In [4]:
import json
import pandas as pd
import spacy
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

nlp = spacy.load("pl_core_news_lg")

In [5]:
with open('data/opisy.json') as f:
    dict_txt = json.load(f)

In [6]:
def get_topics_in_text(opis_dict):
    df = pd.DataFrame()
    for i in range(len(list(dict_txt.keys()))):
        for key, value in dict_txt[list(dict_txt.keys())[i]].items():
            doc = nlp(value)
            list_out = [(t.orth_, t.lemma_, key) for t in doc]
            df = df.append(pd.DataFrame(list_out, columns=['forma','lemat', 'timestamp']))
    df['is_in_topics'] = df['lemat'].apply(lambda x: x in translation)
    return df.reset_index(drop=True)
    

In [ ]:
df = get_topics_in_text(dict_txt)

In [ ]:
df[df['is_in_topics'] == True]['lemat'].value_counts()